In [ ]:
#### Environment Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import plotly.graph_objects as go
import plotly.express as px
import geopandas as gpd
import json
from bokeh.io import output_notebook, show, output_file, curdoc
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.palettes import brewer
from bokeh.models import Slider, HoverTool
from bokeh.models.widgets import Dropdown
from bokeh.models import CustomJS, ColumnDataSource, Select
from bokeh.layouts import widgetbox, row, column
import warnings
warnings.filterwarnings("ignore")

In [ ]:
print('Analysis by Manan Arora')

Website: https://www.aroramanan.in

## Analyzing District Wise Growth of Crop Categories in India

#### Points to Note:

- All the Production Statistics are cummulative for the period - 2013-18 (Source: https://aps.dac.gov.in/APY/Index.htm)

In [ ]:
#### Loading Datasets

In [ ]:
apy_db = pd.read_csv(r'C:/Users/Manan Arora/Desktop/AgriTech Project/Data Driven Business Metrics Project/Data/APY/APY_13_14_to_17_18.csv')
map_dist = gpd.read_file(r'C:\Users\Manan Arora\Desktop\AgriTech Project\Data Driven Business Metrics Project\Data\Shapefiles\District\2011_dist.shp')

In [ ]:
top_crop_cat_dw = apy_db.groupby(['State_code','State_name','District_code','District_name','Crop_category','Crop_code','Crop_name'],sort=True).agg({'Production':'sum'}).reset_index()
top_crop_cat_dw = top_crop_cat_dw.set_index(['Crop_code','Crop_name']).groupby(['State_code','State_name','District_code','District_name','Crop_category'])['Production'].nlargest(1).reset_index()
top_crop_cat_dw.rename(columns={'Production':'Crop_Production'},inplace=True)

In [ ]:
#### Pulses

In [ ]:
top_crop_cat_dw_cer = top_crop_cat_dw[top_crop_cat_dw['Crop_category']=='Pulses']
top_crop_cat_dw_cer.rename(columns={'State_name':'ST_NM','District_name':'DISTRICT'},inplace=True)
top_crop_cat_dw_cer.drop(['District_code','State_code','Crop_category'],axis=1,inplace=True)
apy_db_cer = apy_db[apy_db['Crop_category']=='Pulses']
cer_prod_dw = apy_db_cer.groupby(['State_name','District_name'],sort=True).agg({'Production':'sum'}).reset_index()
cer_prod_dw['Per_Production'] = ((cer_prod_dw['Production']/sum(cer_prod_dw['Production']))*100).round(2)
cer_prod_dw.rename(columns={'Production':'Total_Production'},inplace=True)
cer_prod_dw.rename(columns={'State_name':'ST_NM','District_name':'DISTRICT'},inplace=True)
cer_prod_dw = cer_prod_dw.merge(top_crop_cat_dw_cer,on=['ST_NM','DISTRICT'])
cer_prod_dw['Per_CropinTotal'] = (cer_prod_dw['Crop_Production']/cer_prod_dw['Total_Production'])*100
map_prod_dw = map_dist.merge(cer_prod_dw,on=['DISTRICT','ST_NM'],how = 'left')
map_prod_dw['Total_Production']= map_prod_dw['Total_Production'].fillna(0)
map_prod_dw['Per_Production']= map_prod_dw['Per_Production'].fillna(0)
map_prod_dw['Crop_Production']= map_prod_dw['Crop_Production'].fillna(0)
map_prod_dw['Per_CropinTotal']= map_prod_dw['Per_CropinTotal'].fillna(0)
map_prod_dw_json = json.loads(map_prod_dw.to_json())
map_prod_dw_json = json.dumps(map_prod_dw_json)


#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = map_prod_dw_json)
#Define a sequential multi-hue color palette.
palette = brewer['YlGn'][8]
#Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]
#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 1.5)
#Define custom tick labels for color bar.
tick_labels = {'0': '0', '5': '5%', '10':'10%', '15':'15%', '20':'20%', '25':'25%', '30':'30%','35':'35%', '40': '>40%'}
#hiver
hover = HoverTool(tooltips = [ ('State','@ST_NM'),('District','@DISTRICT'),('% Production', '@Per_Production'),('Top Crop','@Crop_name'),('Share of Top Crop','@Per_CropinTotal')])
#Create color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 450, height = 20,
border_line_color=None,location = (0,0), orientation = 'horizontal', major_label_overrides = tick_labels)
#Create figure object.
p = figure(title = '% Pulses Production District Wise', plot_height = 600 , plot_width = 450, toolbar_location = None, tools=[hover])
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
#Add patch renderer to figure. 
p.patches('xs','ys', source = geosource,fill_color = {'field' :'Per_Production', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)
#Specify figure layout.
p.add_layout(color_bar, 'below')
p.axis.visible = False

#Display figure inline in Jupyter Notebook.
output_notebook()
#Display figure.
show(p)

In [ ]:
#### Cereals

In [ ]:
top_crop_cat_dw_cer = top_crop_cat_dw[top_crop_cat_dw['Crop_category']=='Cereals']
top_crop_cat_dw_cer.rename(columns={'State_name':'ST_NM','District_name':'DISTRICT'},inplace=True)
top_crop_cat_dw_cer.drop(['District_code','State_code','Crop_category'],axis=1,inplace=True)
apy_db_cer = apy_db[apy_db['Crop_category']=='Cereals']
cer_prod_dw = apy_db_cer.groupby(['State_name','District_name'],sort=True).agg({'Production':'sum'}).reset_index()
cer_prod_dw['Per_Production'] = ((cer_prod_dw['Production']/sum(cer_prod_dw['Production']))*100).round(2)
cer_prod_dw.rename(columns={'Production':'Total_Production'},inplace=True)
cer_prod_dw.rename(columns={'State_name':'ST_NM','District_name':'DISTRICT'},inplace=True)
cer_prod_dw = cer_prod_dw.merge(top_crop_cat_dw_cer,on=['ST_NM','DISTRICT'])
cer_prod_dw['Per_CropinTotal'] = (cer_prod_dw['Crop_Production']/cer_prod_dw['Total_Production'])*100
map_prod_dw = map_dist.merge(cer_prod_dw,on=['DISTRICT','ST_NM'],how = 'left')
map_prod_dw['Total_Production']= map_prod_dw['Total_Production'].fillna(0)
map_prod_dw['Per_Production']= map_prod_dw['Per_Production'].fillna(0)
map_prod_dw['Crop_Production']= map_prod_dw['Crop_Production'].fillna(0)
map_prod_dw['Per_CropinTotal']= map_prod_dw['Per_CropinTotal'].fillna(0)
map_prod_dw_json = json.loads(map_prod_dw.to_json())
map_prod_dw_json = json.dumps(map_prod_dw_json)


#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = map_prod_dw_json)
#Define a sequential multi-hue color palette.
palette = brewer['YlGn'][8]
#Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]
#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 1.5)
#Define custom tick labels for color bar.
tick_labels = {'0': '0', '5': '5%', '10':'10%', '15':'15%', '20':'20%', '25':'25%', '30':'30%','35':'35%', '40': '>40%'}
#hiver
hover = HoverTool(tooltips = [ ('State','@ST_NM'),('District','@DISTRICT'),('% Production', '@Per_Production'),('Top Crop','@Crop_name'),('Share of Top Crop','@Per_CropinTotal')])
#Create color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 450, height = 20,
border_line_color=None,location = (0,0), orientation = 'horizontal', major_label_overrides = tick_labels)
#Create figure object.
p = figure(title = '% Cereals Production District Wise', plot_height = 600 , plot_width = 450, toolbar_location = None, tools=[hover])
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
#Add patch renderer to figure. 
p.patches('xs','ys', source = geosource,fill_color = {'field' :'Per_Production', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)
#Specify figure layout.
p.add_layout(color_bar, 'below')
p.axis.visible = False

#Display figure inline in Jupyter Notebook.
output_notebook()
#Display figure.
show(p)

In [ ]:
#### Vegetables

In [ ]:
top_crop_cat_dw_cer = top_crop_cat_dw[top_crop_cat_dw['Crop_category']=='Vegetables']
top_crop_cat_dw_cer.rename(columns={'State_name':'ST_NM','District_name':'DISTRICT'},inplace=True)
top_crop_cat_dw_cer.drop(['District_code','State_code','Crop_category'],axis=1,inplace=True)
apy_db_cer = apy_db[apy_db['Crop_category']=='Vegetables']
cer_prod_dw = apy_db_cer.groupby(['State_name','District_name'],sort=True).agg({'Production':'sum'}).reset_index()
cer_prod_dw['Per_Production'] = ((cer_prod_dw['Production']/sum(cer_prod_dw['Production']))*100).round(2)
cer_prod_dw.rename(columns={'Production':'Total_Production'},inplace=True)
cer_prod_dw.rename(columns={'State_name':'ST_NM','District_name':'DISTRICT'},inplace=True)
cer_prod_dw = cer_prod_dw.merge(top_crop_cat_dw_cer,on=['ST_NM','DISTRICT'])
cer_prod_dw['Per_CropinTotal'] = (cer_prod_dw['Crop_Production']/cer_prod_dw['Total_Production'])*100
map_prod_dw = map_dist.merge(cer_prod_dw,on=['DISTRICT','ST_NM'],how = 'left')
map_prod_dw['Total_Production']= map_prod_dw['Total_Production'].fillna(0)
map_prod_dw['Per_Production']= map_prod_dw['Per_Production'].fillna(0)
map_prod_dw['Crop_Production']= map_prod_dw['Crop_Production'].fillna(0)
map_prod_dw['Per_CropinTotal']= map_prod_dw['Per_CropinTotal'].fillna(0)
map_prod_dw_json = json.loads(map_prod_dw.to_json())
map_prod_dw_json = json.dumps(map_prod_dw_json)


#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = map_prod_dw_json)
#Define a sequential multi-hue color palette.
palette = brewer['YlGn'][8]
#Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]
#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 1.5)
#Define custom tick labels for color bar.
tick_labels = {'0': '0', '5': '5%', '10':'10%', '15':'15%', '20':'20%', '25':'25%', '30':'30%','35':'35%', '40': '>40%'}
#hiver
hover = HoverTool(tooltips = [ ('State','@ST_NM'),('District','@DISTRICT'),('% Production', '@Per_Production'),('Top Crop','@Crop_name'),('Share of Top Crop','@Per_CropinTotal')])
#Create color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 450, height = 20,
border_line_color=None,location = (0,0), orientation = 'horizontal', major_label_overrides = tick_labels)
#Create figure object.
p = figure(title = '% Vegetables Production District Wise', plot_height = 600 , plot_width = 450, toolbar_location = None, tools=[hover])
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
#Add patch renderer to figure. 
p.patches('xs','ys', source = geosource,fill_color = {'field' :'Per_Production', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)
#Specify figure layout.
p.add_layout(color_bar, 'below')
p.axis.visible = False

#Display figure inline in Jupyter Notebook.
output_notebook()
#Display figure.
show(p)

In [ ]:
#### Fruits

In [ ]:
top_crop_cat_dw_cer = top_crop_cat_dw[top_crop_cat_dw['Crop_category']=='Fruits']
top_crop_cat_dw_cer.rename(columns={'State_name':'ST_NM','District_name':'DISTRICT'},inplace=True)
top_crop_cat_dw_cer.drop(['District_code','State_code','Crop_category'],axis=1,inplace=True)
apy_db_cer = apy_db[apy_db['Crop_category']=='Fruits']
cer_prod_dw = apy_db_cer.groupby(['State_name','District_name'],sort=True).agg({'Production':'sum'}).reset_index()
cer_prod_dw['Per_Production'] = ((cer_prod_dw['Production']/sum(cer_prod_dw['Production']))*100).round(2)
cer_prod_dw.rename(columns={'Production':'Total_Production'},inplace=True)
cer_prod_dw.rename(columns={'State_name':'ST_NM','District_name':'DISTRICT'},inplace=True)
cer_prod_dw = cer_prod_dw.merge(top_crop_cat_dw_cer,on=['ST_NM','DISTRICT'])
cer_prod_dw['Per_CropinTotal'] = (cer_prod_dw['Crop_Production']/cer_prod_dw['Total_Production'])*100
map_prod_dw = map_dist.merge(cer_prod_dw,on=['DISTRICT','ST_NM'],how = 'left')
map_prod_dw['Total_Production']= map_prod_dw['Total_Production'].fillna(0)
map_prod_dw['Per_Production']= map_prod_dw['Per_Production'].fillna(0)
map_prod_dw['Crop_Production']= map_prod_dw['Crop_Production'].fillna(0)
map_prod_dw['Per_CropinTotal']= map_prod_dw['Per_CropinTotal'].fillna(0)
map_prod_dw_json = json.loads(map_prod_dw.to_json())
map_prod_dw_json = json.dumps(map_prod_dw_json)


#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = map_prod_dw_json)
#Define a sequential multi-hue color palette.
palette = brewer['YlGn'][8]
#Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]
#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 1.5)
#Define custom tick labels for color bar.
tick_labels = {'0': '0', '5': '5%', '10':'10%', '15':'15%', '20':'20%', '25':'25%', '30':'30%','35':'35%', '40': '>40%'}
#hiver
hover = HoverTool(tooltips = [ ('State','@ST_NM'),('District','@DISTRICT'),('% Production', '@Per_Production'),('Top Crop','@Crop_name'),('Share of Top Crop','@Per_CropinTotal')])
#Create color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 450, height = 20,
border_line_color=None,location = (0,0), orientation = 'horizontal', major_label_overrides = tick_labels)
#Create figure object.
p = figure(title = '% Bananas Production District Wise', plot_height = 600 , plot_width = 450, toolbar_location = None, tools=[hover])
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
#Add patch renderer to figure. 
p.patches('xs','ys', source = geosource,fill_color = {'field' :'Per_Production', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)
#Specify figure layout.
p.add_layout(color_bar, 'below')
p.axis.visible = False

#Display figure inline in Jupyter Notebook.
output_notebook()
#Display figure.
show(p)

In [ ]:
#### Oilseeds

In [ ]:
top_crop_cat_dw_cer = top_crop_cat_dw[top_crop_cat_dw['Crop_category']=='Oilseeds']
top_crop_cat_dw_cer.rename(columns={'State_name':'ST_NM','District_name':'DISTRICT'},inplace=True)
top_crop_cat_dw_cer.drop(['District_code','State_code','Crop_category'],axis=1,inplace=True)
apy_db_cer = apy_db[apy_db['Crop_category']=='Oilseeds']
cer_prod_dw = apy_db_cer.groupby(['State_name','District_name'],sort=True).agg({'Production':'sum'}).reset_index()
cer_prod_dw['Per_Production'] = ((cer_prod_dw['Production']/sum(cer_prod_dw['Production']))*100).round(2)
cer_prod_dw.rename(columns={'Production':'Total_Production'},inplace=True)
cer_prod_dw.rename(columns={'State_name':'ST_NM','District_name':'DISTRICT'},inplace=True)
cer_prod_dw = cer_prod_dw.merge(top_crop_cat_dw_cer,on=['ST_NM','DISTRICT'])
cer_prod_dw['Per_CropinTotal'] = (cer_prod_dw['Crop_Production']/cer_prod_dw['Total_Production'])*100
map_prod_dw = map_dist.merge(cer_prod_dw,on=['DISTRICT','ST_NM'],how = 'left')
map_prod_dw['Total_Production']= map_prod_dw['Total_Production'].fillna(0)
map_prod_dw['Per_Production']= map_prod_dw['Per_Production'].fillna(0)
map_prod_dw['Crop_Production']= map_prod_dw['Crop_Production'].fillna(0)
map_prod_dw['Per_CropinTotal']= map_prod_dw['Per_CropinTotal'].fillna(0)
map_prod_dw_json = json.loads(map_prod_dw.to_json())
map_prod_dw_json = json.dumps(map_prod_dw_json)


#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = map_prod_dw_json)
#Define a sequential multi-hue color palette.
palette = brewer['YlGn'][8]
#Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]
#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 1.5)
#Define custom tick labels for color bar.
tick_labels = {'0': '0', '5': '5%', '10':'10%', '15':'15%', '20':'20%', '25':'25%', '30':'30%','35':'35%', '40': '>40%'}
#hiver
hover = HoverTool(tooltips = [ ('State','@ST_NM'),('District','@DISTRICT'),('% Production', '@Per_Production'),('Top Crop','@Crop_name'),('Share of Top Crop','@Per_CropinTotal')])
#Create color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 450, height = 20,
border_line_color=None,location = (0,0), orientation = 'horizontal', major_label_overrides = tick_labels)
#Create figure object.
p = figure(title = '% Oilseeds Production District Wise', plot_height = 600 , plot_width = 450, toolbar_location = None, tools=[hover])
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
#Add patch renderer to figure. 
p.patches('xs','ys', source = geosource,fill_color = {'field' :'Per_Production', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)
#Specify figure layout.
p.add_layout(color_bar, 'below')
p.axis.visible = False

#Display figure inline in Jupyter Notebook.
output_notebook()
#Display figure.
show(p)

In [ ]:
#### Sugar

In [ ]:
top_crop_cat_dw_cer = top_crop_cat_dw[top_crop_cat_dw['Crop_category']=='Sugar']
top_crop_cat_dw_cer.rename(columns={'State_name':'ST_NM','District_name':'DISTRICT'},inplace=True)
top_crop_cat_dw_cer.drop(['District_code','State_code','Crop_category'],axis=1,inplace=True)
apy_db_cer = apy_db[apy_db['Crop_category']=='Sugar']
cer_prod_dw = apy_db_cer.groupby(['State_name','District_name'],sort=True).agg({'Production':'sum'}).reset_index()
cer_prod_dw['Per_Production'] = ((cer_prod_dw['Production']/sum(cer_prod_dw['Production']))*100).round(2)
cer_prod_dw.rename(columns={'Production':'Total_Production'},inplace=True)
cer_prod_dw.rename(columns={'State_name':'ST_NM','District_name':'DISTRICT'},inplace=True)
cer_prod_dw = cer_prod_dw.merge(top_crop_cat_dw_cer,on=['ST_NM','DISTRICT'])
cer_prod_dw['Per_CropinTotal'] = (cer_prod_dw['Crop_Production']/cer_prod_dw['Total_Production'])*100
map_prod_dw = map_dist.merge(cer_prod_dw,on=['DISTRICT','ST_NM'],how = 'left')
map_prod_dw['Total_Production']= map_prod_dw['Total_Production'].fillna(0)
map_prod_dw['Per_Production']= map_prod_dw['Per_Production'].fillna(0)
map_prod_dw['Crop_Production']= map_prod_dw['Crop_Production'].fillna(0)
map_prod_dw['Per_CropinTotal']= map_prod_dw['Per_CropinTotal'].fillna(0)
map_prod_dw_json = json.loads(map_prod_dw.to_json())
map_prod_dw_json = json.dumps(map_prod_dw_json)


#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = map_prod_dw_json)
#Define a sequential multi-hue color palette.
palette = brewer['YlGn'][8]
#Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]
#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 1.5)
#Define custom tick labels for color bar.
tick_labels = {'0': '0', '5': '5%', '10':'10%', '15':'15%', '20':'20%', '25':'25%', '30':'30%','35':'35%', '40': '>40%'}
#hiver
hover = HoverTool(tooltips = [ ('State','@ST_NM'),('District','@DISTRICT'),('% Production', '@Per_Production'),('Top Crop','@Crop_name'),('Share of Top Crop','@Per_CropinTotal')])
#Create color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 450, height = 20,
border_line_color=None,location = (0,0), orientation = 'horizontal', major_label_overrides = tick_labels)
#Create figure object.
p = figure(title = '% Sugarcane Production District Wise', plot_height = 600 , plot_width = 450, toolbar_location = None, tools=[hover])
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
#Add patch renderer to figure. 
p.patches('xs','ys', source = geosource,fill_color = {'field' :'Per_Production', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)
#Specify figure layout.
p.add_layout(color_bar, 'below')
p.axis.visible = False

#Display figure inline in Jupyter Notebook.
output_notebook()
#Display figure.
show(p)

In [ ]:
#### Fiber Crop

In [ ]:
top_crop_cat_dw_cer = top_crop_cat_dw[top_crop_cat_dw['Crop_category']=='Fiber Crop']
top_crop_cat_dw_cer.rename(columns={'State_name':'ST_NM','District_name':'DISTRICT'},inplace=True)
top_crop_cat_dw_cer.drop(['District_code','State_code','Crop_category'],axis=1,inplace=True)
apy_db_cer = apy_db[apy_db['Crop_category']=='Fiber Crop']
cer_prod_dw = apy_db_cer.groupby(['State_name','District_name'],sort=True).agg({'Production':'sum'}).reset_index()
cer_prod_dw['Per_Production'] = ((cer_prod_dw['Production']/sum(cer_prod_dw['Production']))*100).round(2)
cer_prod_dw.rename(columns={'Production':'Total_Production'},inplace=True)
cer_prod_dw.rename(columns={'State_name':'ST_NM','District_name':'DISTRICT'},inplace=True)
cer_prod_dw = cer_prod_dw.merge(top_crop_cat_dw_cer,on=['ST_NM','DISTRICT'])
cer_prod_dw['Per_CropinTotal'] = (cer_prod_dw['Crop_Production']/cer_prod_dw['Total_Production'])*100
map_prod_dw = map_dist.merge(cer_prod_dw,on=['DISTRICT','ST_NM'],how = 'left')
map_prod_dw['Total_Production']= map_prod_dw['Total_Production'].fillna(0)
map_prod_dw['Per_Production']= map_prod_dw['Per_Production'].fillna(0)
map_prod_dw['Crop_Production']= map_prod_dw['Crop_Production'].fillna(0)
map_prod_dw['Per_CropinTotal']= map_prod_dw['Per_CropinTotal'].fillna(0)
map_prod_dw_json = json.loads(map_prod_dw.to_json())
map_prod_dw_json = json.dumps(map_prod_dw_json)


#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = map_prod_dw_json)
#Define a sequential multi-hue color palette.
palette = brewer['YlGn'][8]
#Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]
#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 1.5)
#Define custom tick labels for color bar.
tick_labels = {'0': '0', '5': '5%', '10':'10%', '15':'15%', '20':'20%', '25':'25%', '30':'30%','35':'35%', '40': '>40%'}
#hiver
hover = HoverTool(tooltips = [ ('State','@ST_NM'),('District','@DISTRICT'),('% Production', '@Per_Production'),('Top Crop','@Crop_name'),('Share of Top Crop','@Per_CropinTotal')])
#Create color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 450, height = 20,
border_line_color=None,location = (0,0), orientation = 'horizontal', major_label_overrides = tick_labels)
#Create figure object.
p = figure(title = '% Fiber Crop Production District Wise', plot_height = 600 , plot_width = 450, toolbar_location = None, tools=[hover])
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
#Add patch renderer to figure. 
p.patches('xs','ys', source = geosource,fill_color = {'field' :'Per_Production', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)
#Specify figure layout.
p.add_layout(color_bar, 'below')
p.axis.visible = False

#Display figure inline in Jupyter Notebook.
output_notebook()
#Display figure.
show(p)

In [ ]:
#### Drug and Narcotics

In [ ]:
top_crop_cat_dw_cer = top_crop_cat_dw[top_crop_cat_dw['Crop_category']=='Drug and Narcotics']
top_crop_cat_dw_cer.rename(columns={'State_name':'ST_NM','District_name':'DISTRICT'},inplace=True)
top_crop_cat_dw_cer.drop(['District_code','State_code','Crop_category'],axis=1,inplace=True)
apy_db_cer = apy_db[apy_db['Crop_category']=='Drug and Narcotics']
cer_prod_dw = apy_db_cer.groupby(['State_name','District_name'],sort=True).agg({'Production':'sum'}).reset_index()
cer_prod_dw['Per_Production'] = ((cer_prod_dw['Production']/sum(cer_prod_dw['Production']))*100).round(2)
cer_prod_dw.rename(columns={'Production':'Total_Production'},inplace=True)
cer_prod_dw.rename(columns={'State_name':'ST_NM','District_name':'DISTRICT'},inplace=True)
cer_prod_dw = cer_prod_dw.merge(top_crop_cat_dw_cer,on=['ST_NM','DISTRICT'])
cer_prod_dw['Per_CropinTotal'] = (cer_prod_dw['Crop_Production']/cer_prod_dw['Total_Production'])*100
map_prod_dw = map_dist.merge(cer_prod_dw,on=['DISTRICT','ST_NM'],how = 'left')
map_prod_dw['Total_Production']= map_prod_dw['Total_Production'].fillna(0)
map_prod_dw['Per_Production']= map_prod_dw['Per_Production'].fillna(0)
map_prod_dw['Crop_Production']= map_prod_dw['Crop_Production'].fillna(0)
map_prod_dw['Per_CropinTotal']= map_prod_dw['Per_CropinTotal'].fillna(0)
map_prod_dw_json = json.loads(map_prod_dw.to_json())
map_prod_dw_json = json.dumps(map_prod_dw_json)


#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = map_prod_dw_json)
#Define a sequential multi-hue color palette.
palette = brewer['YlGn'][8]
#Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]
#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 1.5)
#Define custom tick labels for color bar.
tick_labels = {'0': '0', '5': '5%', '10':'10%', '15':'15%', '20':'20%', '25':'25%', '30':'30%','35':'35%', '40': '>40%'}
#hiver
hover = HoverTool(tooltips = [ ('State','@ST_NM'),('District','@DISTRICT'),('% Production', '@Per_Production'),('Top Crop','@Crop_name'),('Share of Top Crop','@Per_CropinTotal')])
#Create color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 450, height = 20,
border_line_color=None,location = (0,0), orientation = 'horizontal', major_label_overrides = tick_labels)
#Create figure object.
p = figure(title = '% Drug and Narcotics Production District Wise', plot_height = 600 , plot_width = 450, toolbar_location = None, tools=[hover])
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
#Add patch renderer to figure. 
p.patches('xs','ys', source = geosource,fill_color = {'field' :'Per_Production', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)
#Specify figure layout.
p.add_layout(color_bar, 'below')
p.axis.visible = False

#Display figure inline in Jupyter Notebook.
output_notebook()
#Display figure.
show(p)

In [ ]:
#### Dry Fruits

In [ ]:
top_crop_cat_dw_cer = top_crop_cat_dw[top_crop_cat_dw['Crop_category']=='Dry Fruits']
top_crop_cat_dw_cer.rename(columns={'State_name':'ST_NM','District_name':'DISTRICT'},inplace=True)
top_crop_cat_dw_cer.drop(['District_code','State_code','Crop_category'],axis=1,inplace=True)
apy_db_cer = apy_db[apy_db['Crop_category']=='Dry Fruits']
cer_prod_dw = apy_db_cer.groupby(['State_name','District_name'],sort=True).agg({'Production':'sum'}).reset_index()
cer_prod_dw['Per_Production'] = ((cer_prod_dw['Production']/sum(cer_prod_dw['Production']))*100).round(2)
cer_prod_dw.rename(columns={'Production':'Total_Production'},inplace=True)
cer_prod_dw.rename(columns={'State_name':'ST_NM','District_name':'DISTRICT'},inplace=True)
cer_prod_dw = cer_prod_dw.merge(top_crop_cat_dw_cer,on=['ST_NM','DISTRICT'])
cer_prod_dw['Per_CropinTotal'] = (cer_prod_dw['Crop_Production']/cer_prod_dw['Total_Production'])*100
map_prod_dw = map_dist.merge(cer_prod_dw,on=['DISTRICT','ST_NM'],how = 'left')
map_prod_dw['Total_Production']= map_prod_dw['Total_Production'].fillna(0)
map_prod_dw['Per_Production']= map_prod_dw['Per_Production'].fillna(0)
map_prod_dw['Crop_Production']= map_prod_dw['Crop_Production'].fillna(0)
map_prod_dw['Per_CropinTotal']= map_prod_dw['Per_CropinTotal'].fillna(0)
map_prod_dw_json = json.loads(map_prod_dw.to_json())
map_prod_dw_json = json.dumps(map_prod_dw_json)


#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = map_prod_dw_json)
#Define a sequential multi-hue color palette.
palette = brewer['YlGn'][8]
#Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]
#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 1.5)
#Define custom tick labels for color bar.
tick_labels = {'0': '0', '5': '5%', '10':'10%', '15':'15%', '20':'20%', '25':'25%', '30':'30%','35':'35%', '40': '>40%'}
#hiver
hover = HoverTool(tooltips = [ ('State','@ST_NM'),('District','@DISTRICT'),('% Production', '@Per_Production'),('Top Crop','@Crop_name'),('Share of Top Crop','@Per_CropinTotal')])
#Create color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 450, height = 20,
border_line_color=None,location = (0,0), orientation = 'horizontal', major_label_overrides = tick_labels)
#Create figure object.
p = figure(title = '% Dry Fruits Production District Wise', plot_height = 600 , plot_width = 450, toolbar_location = None, tools=[hover])
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
#Add patch renderer to figure. 
p.patches('xs','ys', source = geosource,fill_color = {'field' :'Per_Production', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)
#Specify figure layout.
p.add_layout(color_bar, 'below')
p.axis.visible = False

#Display figure inline in Jupyter Notebook.
output_notebook()
#Display figure.
show(p)

In [ ]:
#### Spices

In [ ]:
top_crop_cat_dw_cer = top_crop_cat_dw[top_crop_cat_dw['Crop_category']=='Spices']
top_crop_cat_dw_cer.rename(columns={'State_name':'ST_NM','District_name':'DISTRICT'},inplace=True)
top_crop_cat_dw_cer.drop(['District_code','State_code','Crop_category'],axis=1,inplace=True)
apy_db_cer = apy_db[apy_db['Crop_category']=='Spices']
cer_prod_dw = apy_db_cer.groupby(['State_name','District_name'],sort=True).agg({'Production':'sum'}).reset_index()
cer_prod_dw['Per_Production'] = ((cer_prod_dw['Production']/sum(cer_prod_dw['Production']))*100).round(2)
cer_prod_dw.rename(columns={'Production':'Total_Production'},inplace=True)
cer_prod_dw.rename(columns={'State_name':'ST_NM','District_name':'DISTRICT'},inplace=True)
cer_prod_dw = cer_prod_dw.merge(top_crop_cat_dw_cer,on=['ST_NM','DISTRICT'])
cer_prod_dw['Per_CropinTotal'] = (cer_prod_dw['Crop_Production']/cer_prod_dw['Total_Production'])*100
map_prod_dw = map_dist.merge(cer_prod_dw,on=['DISTRICT','ST_NM'],how = 'left')
map_prod_dw['Total_Production']= map_prod_dw['Total_Production'].fillna(0)
map_prod_dw['Per_Production']= map_prod_dw['Per_Production'].fillna(0)
map_prod_dw['Crop_Production']= map_prod_dw['Crop_Production'].fillna(0)
map_prod_dw['Per_CropinTotal']= map_prod_dw['Per_CropinTotal'].fillna(0)
map_prod_dw_json = json.loads(map_prod_dw.to_json())
map_prod_dw_json = json.dumps(map_prod_dw_json)


#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = map_prod_dw_json)
#Define a sequential multi-hue color palette.
palette = brewer['YlGn'][8]
#Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]
#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 1.5)
#Define custom tick labels for color bar.
tick_labels = {'0': '0', '5': '5%', '10':'10%', '15':'15%', '20':'20%', '25':'25%', '30':'30%','35':'35%', '40': '>40%'}
#hiver
hover = HoverTool(tooltips = [ ('State','@ST_NM'),('District','@DISTRICT'),('% Production', '@Per_Production'),('Top Crop','@Crop_name'),('Share of Top Crop','@Per_CropinTotal')])
#Create color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 450, height = 20,
border_line_color=None,location = (0,0), orientation = 'horizontal', major_label_overrides = tick_labels)
#Create figure object.
p = figure(title = '% Spices Production District Wise', plot_height = 600 , plot_width = 450, toolbar_location = None, tools=[hover])
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
#Add patch renderer to figure. 
p.patches('xs','ys', source = geosource,fill_color = {'field' :'Per_Production', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)
#Specify figure layout.
p.add_layout(color_bar, 'below')
p.axis.visible = False

#Display figure inline in Jupyter Notebook.
output_notebook()
#Display figure.
show(p)

In [ ]:
def json_data(category):
    top_crop_cat_dw_cer = top_crop_cat_dw[top_crop_cat_dw['Crop_category']==category]
    top_crop_cat_dw_cer.rename(columns={'State_name':'ST_NM','District_name':'DISTRICT'},inplace=True)
    top_crop_cat_dw_cer.drop(['District_code','State_code','Crop_category'],axis=1,inplace=True)
    apy_db_cer = apy_db[apy_db['Crop_category']==category]
    cer_prod_dw = apy_db_cer.groupby(['State_name','District_name'],sort=True).agg({'Production':'sum'}).reset_index()
    cer_prod_dw['Per_Production'] = ((cer_prod_dw['Production']/sum(cer_prod_dw['Production']))*100).round(2)
    cer_prod_dw.rename(columns={'Production':'Total_Production'},inplace=True)
    cer_prod_dw.rename(columns={'State_name':'ST_NM','District_name':'DISTRICT'},inplace=True)
    cer_prod_dw = cer_prod_dw.merge(top_crop_cat_dw_cer,on=['ST_NM','DISTRICT'])
    cer_prod_dw['Per_CropinTotal'] = (cer_prod_dw['Crop_Production']/cer_prod_dw['Total_Production'])*100
    map_prod_dw = map_dist.merge(cer_prod_dw,on=['DISTRICT','ST_NM'],how = 'left')
    map_prod_dw['Total_Production']= map_prod_dw['Total_Production'].fillna(0)
    map_prod_dw['Per_Production']= map_prod_dw['Per_Production'].fillna(0)
    map_prod_dw['Crop_Production']= map_prod_dw['Crop_Production'].fillna(0)
    map_prod_dw['Per_CropinTotal']= map_prod_dw['Per_CropinTotal'].fillna(0)
    map_prod_dw_json = json.loads(map_prod_dw.to_json())
    map_prod_dw_json = json.dumps(map_prod_dw_json)
    
    return map_prod_dw_json


#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = json_data('Pulses'))
#Define a sequential multi-hue color palette.
palette = brewer['YlGn'][8]
#Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]
#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 1.5)
#Define custom tick labels for color bar.
tick_labels = {'0': '0', '5': '5%', '10':'10%', '15':'15%', '20':'20%', '25':'25%', '30':'30%','35':'35%', '40': '>40%'}
#hiver
hover = HoverTool(tooltips = [ ('State','@ST_NM'),('District','@DISTRICT'),('% Production', '@Per_Production'),('Top Crop','@Crop_name'),('Share of Top Crop','@Per_CropinTotal')])
#Create color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 450, height = 20,
border_line_color=None,location = (0,0), orientation = 'horizontal', major_label_overrides = tick_labels)
#Create figure object.
p = figure(title = '% Pulses Production District Wise', plot_height = 600 , plot_width = 450, toolbar_location = None, tools=[hover])
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
#Add patch renderer to figure. 
p.patches('xs','ys', source = geosource,fill_color = {'field' :'Per_Production', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)
#Specify figure layout.
p.add_layout(color_bar, 'below')
p.axis.visible = False

menu = [("Pulses", "Pulses"), ("Cereals", "Cereals"), ("Fruits", "Fruits"),("Vegetables",'Vegetables'),('Sugar','Sugar'),
        ('Oilseeds','Oilseeds'),('Spices','Spices'),('Dry Fruits','Dry Fruits'),('Fiber Crop','Fiber Crop'),('Drug and Narcotics','Drug and Narcotics')]
dropdown = Dropdown(label="Select the Crop Category", menu=menu)

def update_plot(event):
    category = event.item
    new_data = json_data(category)
    geosource.geojson = new_data
    p.title.text = '% {} Production District Wise'.format(category)
    
dropdown.on_click(update_plot)
# Make a column layout of widgetbox(slider) and plot, and add it to the current document
layout = column(dropdown,p)
curdoc().add_root(layout)

#Display figure inline in Jupyter Notebook.
output_notebook()
#Display figure.
show(layout)